[https://github.com/victorsimrbt/Chess-Deep-Learning/blob/main/Chess%20Ai%20Trainer%20v4.ipynb](https://github.com/victorsimrbt/Chess-Deep-Learning/blob/main/Chess%20Ai%20Trainer%20v4.ipynb)

In [1]:
import os
import chess
import numpy as np
import pandas as pd
import time

In [2]:
from tensorflow.keras import callbacks, optimizers
from tensorflow.python.keras.layers import (LSTM, BatchNormalization, Dense, Dropout, Flatten,
                          TimeDistributed)
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, load_model, model_from_json
from IPython.display import clear_output
from matplotlib import pyplot as plt

In [3]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [4]:
df = pd.read_csv('games.csv')
data = df['moves'].tolist()[:500]

In [5]:
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}

In [6]:
def make_matrix(board): 
    pgn = board.epd()
    foo = []  
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo

In [7]:
def translate(matrix,chess_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows

In [8]:
def data_setup():
    X = []
    y = []
    for game in range(len(data)):
        data[game] = data[game].split()
        board = chess.Board()
        for move in range(len(data[game])):
            legal_moves = str(board.legal_moves)[36:-2].replace(',','').split()
            matrix = make_matrix(board.copy())
            translated = translate(matrix,chess_dict)
            X.append(translated)
            board.push_san(data[game][move])
            y.append(legal_moves.index(data[game][move])/len(legal_moves))
    return X,y

In [9]:
def initialize_network():
    model = Sequential()
    model.add(Conv2D(filters=10, kernel_size=1, activation='relu', input_shape=(8,8,12)))
    model.add(MaxPooling2D(pool_size=2, strides=None))
    model.add(Conv2D(filters=10, kernel_size=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=None))
    model.add(Conv2D(filters=10, kernel_size=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=None))
    model.add(Conv2D(filters=10, kernel_size=1, activation='relu'))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(1,activation = 'sigmoid'))
    return model

In [10]:
print('Preparing Data...')
start = time.time()
X,y = data_setup()
one_game_time = time.time()-start
X = np.array(X)
X_test = X[:int(len(X)*0.1)]
X_train = X[int(len(X)*0.1):]
y_test = y[:int(len(X)*0.1)]
y_train = y[int(len(X)*0.1):]
print('Initalizing Network...')
model = initialize_network()
print('Compiling Network...')

Preparing Data...


ValueError: 'Ke7' is not in list

In [ ]:
print('Preparing Data...')
start = time.time()
X,y = data_setup()
one_game_time = time.time()-start
X = np.array(X)
X_test = X[:int(len(X)*0.1)]
X_train = X[int(len(X)*0.1):]
y_test = y[:int(len(X)*0.1)]
y_train = y[int(len(X)*0.1):]
print('Initalizing Network...')
model = initialize_network()
print('Compiling Network...')

In [ ]:
chess_board = chess.Board()
for i in range(100):
    matrix = make_matrix(chess_board)
    board = translate(matrix,chess_dict)
    board = np.array(board)
    board = np.reshape(board,(1,8,8,12))
    pred = model.predict(board)
    legal_moves = str(chess_board.legal_moves)[36:-2].replace(',','').split()
    index = int(round((len(legal_moves)*pred)[0][0]))
    chess_board.push_san(legal_moves[index])
    if chess_board.is_checkmate():
        break
chess_board